In [1]:
import sys
sys.path.append('../')  # Ajustez le chemin selon la structure de votre dossier


In [2]:
from dataclasses import dataclass
from omegaconf import DictConfig

from lightning.pytorch import Trainer
from lightning.pytorch.loggers import MLFlowLogger

from src.models.object_detector import FasterRCNNModule
from src.data.data_module import GrotiusDataModule
from src.callbacks.mlflow_callback import MLFlowModelRegistryCallback

In [3]:
# ~~~ Configuration ~~~
@dataclass
class Config:
    data_dir: str = (
        "/home/olivier/projet/pi/monorepo/doc-analyzer/model-experiment/data"
    )
    batch_size: int = 2
    num_workers: int = 4
    shuffle: bool = True
    weights: str = "FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT"
    dataset_files = ("CRH.yaml", "fasterRCNN_v2.yaml")
    experiment_name: str = "doc-analyzer-v1.0"
    logger_uri: str = "file:./mlruns"
    max_epochs: int = 1

config = Config()

In [4]:
# ~~~ Data Preparation ~~~
data_module = GrotiusDataModule(
    config.data_dir, config.batch_size, config.num_workers, config.shuffle
)
data_module.setup()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
# ~~~ Model Initialization ~~~
model = FasterRCNNModule(config.weights)

In [6]:
# ~~~ Logger ~~~
mlf_logger = MLFlowLogger(
    experiment_name=config.experiment_name,
    tracking_uri=config.logger_uri,
    log_model=True,
)

In [7]:

# ~~~ Training ~~~
trainer = Trainer(
    limit_train_batches=100,
    max_epochs=config.max_epochs,
    logger=mlf_logger,
    fast_dev_run=False,
    callbacks=[MLFlowModelRegistryCallback(config.experiment_name)],
)
trainer.fit(
    model,
    train_dataloaders=data_module,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                  | Params
------------------------------------------------
0 | model | FasterRCNN            | 43.3 M
1 | iou   | IntersectionOverUnion | 0     
------------------------------------------------
43.0 M    Trainable params
225 K     Non-trainable params
43.3 M    Total params
173.066   Total estimated model params size (MB)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Epoch 0: 100%|██████████| 64/64 [00:41<00:00,  1.54it/s, v_num=ad36, train_loss_step=0.744, val_loss_step=0.878, val_loss_epoch=0.802, train_loss_epoch=1.240]

`Trainer.fit` stopped: `max_epochs=1` reached.
2024/05/05 15:59:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp6fkg6ore/model/data, flavor: pytorch). Fall back to return ['torch==2.2.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 


Model doc-analyzer-v1.0 version 4 created in MLFlow Model Registry.
Epoch 0: 100%|██████████| 64/64 [00:46<00:00,  1.38it/s, v_num=ad36, train_loss_step=0.744, val_loss_step=0.878, val_loss_epoch=0.802, train_loss_epoch=1.240]


In [8]:
trainer.test(model, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Testing DataLoader 0: 100%|██████████| 21/21 [00:04<00:00,  5.01it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           iou              0.03561413660645485
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'iou': 0.03561413660645485}]